## Lab 3 on Practical Machine Learning and Deep Learning

Author: Danis Alukaev <br>
Group: B19-DS-01 <br>
Email: d.alukaev@innopolis.university

# German traffic sign recognition 

Imagine you are working in self-driving project as a machine learning engineer. One of the important problems for self-driving cars is to follow traffic signs. A car moves and collects video from the front camera. In each separate image you could localize the signs and after that understand what sign did we find.

We can assume that someone else is responsible for localizing and tracking signs, and your task is to do the classification part.

In today's lab you will be asked to develop a traffic sign recognition system. It should take a sign image and classify which of 43 signs it belongs to.

In this problem you will be given a benchmark dataset The German Traffic Sign Recognition Benchmark: A multi-class classification competition.

Overview
1. multi-class classification problem
2. More than 40 classes
3. More than 50,000 images in total
4. Large, lifelike database
5. Reliable ground-truth data due to semi-automatic annotation
Physical traffic sign instances are unique within the dataset
(i.e., each real-world traffic sign only occurs once)

In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import pandas as pd
import torch
import torch.optim as optim
from torch.utils.data import DataLoader , Dataset
from PIL import Image
from collections import Counter
import torchvision
from torchvision import transforms
import torch.utils

/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


I have downloaded dataset from [Kaggle Competition](https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign) and trained model locally on my machine.

In [ ]:
### GET THE DATA FROM DRIVE

# from google.colab import drive
# drive.mount('/content/drive')
# !unzip 'drive/MyDrive/GTSRB/GTRSB.zip'

!unzip 'GTSRB.zip'

I put these hyperparameters in separate cell and increased image size to 299.

In [2]:
BATCH_SIZE = 70
IMAGE_SIZE = 299
EPOCHS = 10
NB_CLASS = 43

In [3]:
### The dataset contains Train and Test folders + Train.csv and Test.csv
### TASK : load data using csv file 

def get_data(csv_data):
    images, labels = [], []
    ### open csv file
    df = pd.read_csv(csv_data)

    ### iterate the csv , for each row get image from the 'Path' column and get label from the column 'ClassId'
    for idx, row in df.iterrows():
        images.append(np.array(Image.open(row['Path'])))
        labels.append(row['ClassId'])
    
    return images, labels

### Next we need to build a custom Dataset in Pytorch, this dataset will be given after that to the Dataloader 

class CustomDataset(Dataset):
    def __init__(self, images, labels, transforms=None):
        self.labels = labels
        self.images = images 
        self.transform = transforms

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label


train_transform = transforms.Compose([
    ### Define augmentations 
    transforms.ToTensor(),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE))
    ### Don't forget to convert to tensor
])

test_transform = transforms.Compose([
   ### put code here
   transforms.ToTensor(),
   transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
])

x_train, y_train = get_data('Train.csv')
train_data = CustomDataset(x_train, y_train, transforms=train_transform)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

x_test, y_test = get_data('Test.csv')
test_data = CustomDataset(x_test, y_test, transforms=test_transform)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

print("Train data %.d  Test data %.d "%(len(train_loader.dataset),len(test_loader.dataset)))

Train data 39209  Test data 12630 


In the following cell I have implemented simple CNN and used InceptionV3 for fine-tuning. Note that there are pairs of optimizers and schedulers compared to original notebook.

In [4]:
import torch.nn as nn
import torch.nn.functional as F


### If you run on GPU you must load it to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### TASK : Define 2 models the first a simple CNN and the second choose any pretrained model

class CNN_model(nn.Module):

    def __init__(self, nb_class):
        super(CNN_model, self).__init__()
        ### define layers here
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3))
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3))

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128) 

        self.fc1 = nn.Linear(156800, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, nb_class)

        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.bn1(self.pool(F.relu(self.conv1(x))))
        x = self.bn2(self.pool(F.relu(self.conv2(x))))
        x = self.bn3(self.pool(F.relu(self.conv3(x))))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

class Pretrained_model(nn.Module):
    
    def __init__(self, nb_class):
        super(Pretrained_model, self).__init__()

        ### from torchvision.models choose a pretrained model
        # I selected InceptionV3 as pretrained model.

        model = torchvision.models.inception_v3(pretrained=True)
        model.AuxLogits.fc = nn.Linear(768, nb_class)
        model.fc = nn.Linear(2048, nb_class)
        self.model = model

    def forward(self, x):
        x = self.model(x)
        if self.training:
            x, _ = x
        return x


model = CNN_model(NB_CLASS).to(device)
model_ft = Pretrained_model(NB_CLASS).to(device)

# Loss function here
loss_fn = nn.CrossEntropyLoss()
# Optimizer here
optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
# Scheduler here
schedule = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
schedule_ft = optim.lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

In [5]:
def count_correct(y_pred, y):
    ### task : make a function that counts correctly classified samples
    _, predicted_classes = y_pred.max(1)
    predicted_classes = predicted_classes.cpu().detach().numpy()
    y = y.cpu().detach().numpy()
    acc = (y == predicted_classes).sum()
    return acc

def train(model, train_loader, loss_fn, optimizer,scheduler,device):
    epoch_accuracy = 0
    epoch_correct = 0
    running_loss = 0
    total = 0
    correct = 0

    model.train()
    for i, data in enumerate(train_loader):
        # task : get the images and labels and don't forget to load to device (2 lines)
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        # zero the parameter gradients, it's a must 
        optimizer.zero_grad()
        outputs = model(images)

        # task : get the loss 
        loss = loss_fn(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()* images.size(0)
        total += images.size(0)
        correct += count_correct(outputs, labels)
        epoch_accuracy = correct / total
        
        # task : print statistics here (once in 100 iterations)
        if i % 100 == 0:
            print(f"Iteration {i}: loss={running_loss / total:.3f} acc={epoch_accuracy:.3f}")

        
def evaluate(model, loader, criterion, device):
    epoch_loss = 0
    epoch_acc = 0
    total = 0
    correct = 0
    
    # Evaluate the model
    model.eval()
    with torch.no_grad():
        for (images, labels) in loader:
            ## task :get images and labels and add to device 
            images, labels = images.to(device), labels.to(device)
            
            output = model(images)
            epoch_loss += criterion(output, labels)
            
            total += labels.shape[0]

            # Count correctly classified samples
            correct += count_correct(output, labels)
            
            ### task : print statistics
        print(f"Validation: loss={epoch_loss / total:.3f} acc={correct / total:.3f}")
    

### Train and Test the designed CNN

In [6]:
for epoch in range(EPOCHS):
    print(f"Epoch #{epoch + 1}")
    train(model, train_loader, loss_fn, optimizer, schedule, device)
    evaluate(model, test_loader, loss_fn, device)

Epoch #1
Iteration 0: loss=3.761 acc=0.043
Iteration 100: loss=3.778 acc=0.229
Iteration 200: loss=2.963 acc=0.336
Iteration 300: loss=2.475 acc=0.418
Iteration 400: loss=2.096 acc=0.494
Iteration 500: loss=1.823 acc=0.555
Validation: loss=0.019 acc=0.692
Epoch #2
Iteration 0: loss=0.797 acc=0.786
Iteration 100: loss=0.555 acc=0.862
Iteration 200: loss=0.479 acc=0.881
Iteration 300: loss=0.430 acc=0.895
Iteration 400: loss=0.408 acc=0.900
Iteration 500: loss=0.389 acc=0.904
Validation: loss=0.015 acc=0.799
Epoch #3
Iteration 0: loss=0.167 acc=0.957
Iteration 100: loss=0.246 acc=0.940
Iteration 200: loss=0.204 acc=0.949
Iteration 300: loss=0.185 acc=0.953
Iteration 400: loss=0.183 acc=0.954
Iteration 500: loss=0.183 acc=0.953
Validation: loss=0.011 acc=0.811
Epoch #4
Iteration 0: loss=0.057 acc=0.986
Iteration 100: loss=0.094 acc=0.976
Iteration 200: loss=0.112 acc=0.973
Iteration 300: loss=0.150 acc=0.965
Iteration 400: loss=0.138 acc=0.967
Iteration 500: loss=0.131 acc=0.968
Validatio

Custom model achieved `85.2%`. Not bad at all! Let's see how fine-tuned model will perform.

### Fine-tune and Test InceptionV3 (for bonus point)

In [7]:
for epoch in range(EPOCHS):
    print(f"Epoch #{epoch + 1}")
    train(model_ft, train_loader, loss_fn, optimizer_ft, schedule_ft, device)
    evaluate(model_ft, test_loader, loss_fn, device)

Epoch #1
Iteration 0: loss=3.802 acc=0.043
Iteration 100: loss=0.722 acc=0.809
Iteration 200: loss=0.428 acc=0.889
Iteration 300: loss=0.315 acc=0.918
Iteration 400: loss=0.251 acc=0.935
Iteration 500: loss=0.212 acc=0.945
Validation: loss=0.001 acc=0.978
Epoch #2
Iteration 0: loss=0.032 acc=0.986
Iteration 100: loss=0.068 acc=0.982
Iteration 200: loss=0.063 acc=0.983
Iteration 300: loss=0.051 acc=0.987
Iteration 400: loss=0.043 acc=0.989
Iteration 500: loss=0.038 acc=0.990
Validation: loss=0.002 acc=0.967
Epoch #3
Iteration 0: loss=0.032 acc=0.986
Iteration 100: loss=0.014 acc=0.997
Iteration 200: loss=0.020 acc=0.995
Iteration 300: loss=0.030 acc=0.993
Iteration 400: loss=0.028 acc=0.994
Iteration 500: loss=0.027 acc=0.994
Validation: loss=0.002 acc=0.971
Epoch #4
Iteration 0: loss=0.013 acc=1.000
Iteration 100: loss=0.057 acc=0.987
Iteration 200: loss=0.034 acc=0.992
Iteration 300: loss=0.026 acc=0.994
Iteration 400: loss=0.026 acc=0.993
Iteration 500: loss=0.027 acc=0.993
Validatio

Wow, we achieved an accuracy of `98.3%` on test set, that's cool!